In [1]:
%matplotlib qt5
import numpy as np
from numpy import linalg as la
import matplotlib.pylab as plt
from scipy.optimize import fsolve
import math
import statsmodels.api as sm
from scipy.stats import norm
import seaborn as sns
from scipy import stats
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

In [2]:
import scipy
from functools import partial
def odeIntegral(x,t,J,I=0):
    x = np.squeeze(x)
    x = np.reshape(x,(len(x),1))
    # dxdt = -x+J@np.tanh(x)#+I[0]
    dxdt = -x+J@x+I[0]
    return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)

In [4]:
''' Parameters used for all networks (reciprocal and chain)'''
### get the current path location and read the data file
import os
strr= os.getcwd()
print(strr)

### define the network parameters of the adjacency matrix
N  = 1500
Kt = int(N*0.2)
J = 1/np.sqrt(N)*0.5  ### TODO: make sure this scalar with David&Stefano's paper
J = 0.00325
ntau   = 10
trials = 30+6
tau_series = np.linspace(0,0.15,ntau)#
htau = tau_series[1]-tau_series[0]
### simulation using the low-rank framework
firing_rateeq = np.zeros((trials,ntau,N))

g, gamma =  6, 1/4.0
NE = int(N/(1+gamma))
NI = N-NE#NE*gamma
N  = NE+NI ### update 
ALPHAE, ALPHAI = NE/N, NI/N
KE, KI = int(Kt/(1+gamma)), int(Kt/(1+gamma)*gamma) ### fixed out-degree
ce, ci = KE/NE, KI/NI
print('ce and ci:',ce,ci)
### assert that the differences between ce and ci are smaller than epsilon
epsilon = 1E-2
assert np.abs(ce-ci)<epsilon
# assert ce==ci
c = ce
ji,je = g*J,J 
### define the network parameters of the diluted Gaussian matrix 
ge, gi = np.sqrt(je**2*c*(1-c)*N), np.sqrt(ji**2*c*(1-c)*N) 
hat_sigmae, hat_sigmai = np.sqrt(c*(1-c)), np.sqrt(c*(1-c))### standard deviation of the adjacency matrix
sigmae,sigmai = np.sqrt(c*(1-c)*J**2*N), np.sqrt(c*(1-c)*(-g*J)**2*N)### with magnitude of the coupling
JE,JI = je*c*NE, ji*c*NI 
lambda0 = JE-JI 
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 


e:\Dropbox\DailyWork\Allen_project\Preparation_Paper_Figures\PRX_prune\PRX_numeric\LocalChainMotifs_ParadoxicalEffect
ce and ci: 0.2 0.2
eigvJ0: (-0.3899999999999837+0j)  theory: -0.39000000000000024
JE: 0.7799999999999999 JI: 1.1700000000000002


In [4]:
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 
## TEST THE EIGENVALUES OF THE MEAN MATRIX 
eigvJ0, eigvecJ0 = la.eig(Jbar)
print('eigvJ0:',eigvJ0[0],' theory:',JE-JI)

eigvJ0: (-0.3899999999999837+0j)  theory: -0.39000000000000024
DeltaZ2E,DeltaZ2I: -1.690000000000001e-05 0.00010140000000000003


In [5]:
import scipy
from functools import partial
def odeIntegral(x,t,J,I=0):
    x = np.squeeze(x)
    x = np.reshape(x,(len(x),1))
    # dxdt = -x+J@np.tanh(x)#+I[0]
    dxdt = -x+J@x+I[0]
    return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)
shiftx = 1.5
def odeIntegralP(x,t,J,I=0):
	x = np.squeeze(x)
	x = np.reshape(x,(len(x),1))
	# print('size:',np.shape(x),np.shape(J@np.tanh(x)))
	dxdt = -x+J@(1.0+np.tanh(x-shiftx))
	return np.squeeze(dxdt)
def odesimulationP(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegralP,J=Jpt,I=I),xinit,t)

In [6]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
Ipert = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
# Ipert[NE:]=0 ### only perturb the excitatory population
Ipert[:NE]=0 ### only perturb the inhibitory population
tt = np.linspace(0,200,1000)

In [7]:
### generate i.i.d. s
def randbin(M,N,P):  
    return np.random.choice([0, 1], size=(M,N), p=[P, 1-P])

[0.01853083 0.0185637 ]


In [8]:
### simulation using the low-rank framework
firing_rateEFF     = np.zeros((ntau,ntau*2-1,N))
firing_rateEFFpert = np.zeros((ntau,ntau*2-1,N))

z2 = np.zeros((N,N))
for it, tau in enumerate(tau_series):# outer
    for iti,taui in enumerate(tau_series[::-1]):
        print('it,iti:',it,iti)
        ### compute the square of the random connectivity 
        Z2E = N*J**2*hat_sigmae**2*tau*ALPHAE-N*g*J**2*hat_sigmae*hat_sigmai*taui*ALPHAI
        Z2I = -N*g*J**2*hat_sigmae*hat_sigmai*taui*ALPHAE+N*g**2*J**2*hat_sigmai**2*tau*ALPHAI
        z2[:,:NE], z2[:,NE:] = Z2E, Z2I
        J_EFF = Jbar.copy()+z2.copy()

        ### generate J connectivity matrix    
        '''### full rank simulation'''
        xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
        xc_temporal = odesimulation(tt, xinit, J_EFF, Inp)
        firing_rateEFF[it,iti,:] = xc_temporal[-1,:].copy()
        '''### perturbation '''
        xpert = xc_temporal[-1,:].copy()
        xpert = xpert.reshape(-1,1)
        dtt =tt[1]-tt[0]
        for ttt in range(len(tt)):
            delta_x= -xpert + J_EFF@xpert.reshape(-1,1)+Ipert.reshape(-1,1)+Inp.reshape(-1,1)
            xpert = delta_x*dtt+xpert 
        firing_rateEFFpert[it,iti,:] = xpert.copy().squeeze()
    
    for iti,taui in enumerate(-tau_series[1:]):
        print('it,iti:',it,iti+len(tau_series))
        ### compute the square of the random connectivity 
        Z2E = N*J**2*hat_sigmae**2*tau*ALPHAE-N*g*J**2*hat_sigmae*hat_sigmai*taui*ALPHAI
        Z2I = -N*g*J**2*hat_sigmae*hat_sigmai*taui*ALPHAE+N*g**2*J**2*hat_sigmai**2*tau*ALPHAI
        z2[:,:NE], z2[:,NE:] = Z2E, Z2I
        J_EFF = Jbar.copy()+z2.copy()

        ### generate J connectivity matrix    
        '''### full rank simulation'''
        xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
        xc_temporal = odesimulation(tt, xinit, J_EFF, Inp)
        firing_rateEFF[it,iti+len(tau_series),:] = xc_temporal[-1,:].copy()
        '''### perturbation '''
        xpert = xc_temporal[-1,:].copy()
        xpert = xpert.reshape(-1,1)
        dtt =tt[1]-tt[0]
        for ttt in range(len(tt)):
            delta_x= -xpert + J_EFF@xpert.reshape(-1,1)+Ipert.reshape(-1,1)+Inp.reshape(-1,1)
            xpert = delta_x*dtt+xpert 
        firing_rateEFFpert[it,iti+len(tau_series),:] = xpert.copy().squeeze()

it,iti: 0 0
it,iti: 0 1
it,iti: 0 2
it,iti: 0 3
it,iti: 0 4
it,iti: 0 5
it,iti: 0 6
it,iti: 0 7
it,iti: 0 8
it,iti: 0 9
it,iti: 0 10
it,iti: 0 11
it,iti: 0 12
it,iti: 0 13
it,iti: 0 14
it,iti: 0 15
it,iti: 0 16
it,iti: 0 17
it,iti: 0 18
it,iti: 1 0
it,iti: 1 1
it,iti: 1 2
it,iti: 1 3
it,iti: 1 4
it,iti: 1 5
it,iti: 1 6
it,iti: 1 7
it,iti: 1 8
it,iti: 1 9
it,iti: 1 10
it,iti: 1 11
it,iti: 1 12
it,iti: 1 13
it,iti: 1 14
it,iti: 1 15
it,iti: 1 16
it,iti: 1 17
it,iti: 1 18
it,iti: 2 0
it,iti: 2 1
it,iti: 2 2
it,iti: 2 3
it,iti: 2 4
it,iti: 2 5
it,iti: 2 6
it,iti: 2 7
it,iti: 2 8
it,iti: 2 9
it,iti: 2 10
it,iti: 2 11
it,iti: 2 12
it,iti: 2 13
it,iti: 2 14
it,iti: 2 15
it,iti: 2 16
it,iti: 2 17
it,iti: 2 18
it,iti: 3 0
it,iti: 3 1
it,iti: 3 2
it,iti: 3 3
it,iti: 3 4
it,iti: 3 5
it,iti: 3 6
it,iti: 3 7
it,iti: 3 8
it,iti: 3 9
it,iti: 3 10
it,iti: 3 11
it,iti: 3 12
it,iti: 3 13
it,iti: 3 14
it,iti: 3 15
it,iti: 3 16
it,iti: 3 17
it,iti: 3 18
it,iti: 4 0
it,iti: 4 1
it,iti: 4 2
it,iti: 4 3
it,i

In [9]:
### simulation using the low-rank framework
tau_inner = []
for it, tau in enumerate(tau_series[:1]):# outer
    for iti,taui in enumerate(tau_series[::-1]):
        tau_inner.append(taui)
    
    for iti,taui in enumerate(-tau_series[1:]):
        tau_inner.append(taui)

In [10]:
### 
meanfr_eff      = np.zeros((ntau,2*ntau-1,2))
meanfr_eff_pert = np.zeros((ntau,2*ntau-1,2))
meanfr_eff[:,:,0] = np.mean(firing_rateEFF[:,:,:NE],axis=2)
meanfr_eff[:,:,1] = np.mean(firing_rateEFF[:,:,NE:],axis=2)
### same for pydll Creates ()
meanfr_eff_pert[:,:,0] = np.mean(firing_rateEFFpert[:,:,:NE],axis=2)
meanfr_eff_pert[:,:,1] = np.mean(firing_rateEFFpert[:,:,NE:],axis=2)
### numerical response function 
response_func_eff = np.zeros((ntau,ntau*2-1,2))
for it in range(ntau):
    for iti in range(ntau*2-1):
        response_func_eff[it,iti,0]=(meanfr_eff_pert[it,iti,0]-meanfr_eff[it,iti,0])/Ipert[-1]
        response_func_eff[it,iti,1]=(meanfr_eff_pert[it,iti,1]-meanfr_eff[it,iti,1])/Ipert[-1]

In [ ]:
response_func_eff_2D_binary = response_func_eff.copy()
response_func_eff_2D_binary[response_func_eff_2D_binary>0]=1
response_func_eff_2D_binary[response_func_eff_2D_binary<0]=-1
# find the boundary of -1 and 1 
boundary = np.zeros((ntau,2))
for it in range(ntau):
    for ipop in range(2):
        boundary[it,ipop] = np.where(response_func_eff_2D_binary[it,:,ipop]==-1)[0][0]
print(boundary)

In [24]:
## imshow the response function response_func_eff_2D_binary
fig,ax=plt.subplots(figsize=(8,6))
cmap = sns.color_palette("coolwarm", as_cmap=True)
im=ax.imshow(response_func_eff[:,:,1],cmap=cmap,alpha=0.25,vmax=0.2,vmin=-0.2)
plt.colorbar(im)
plt.show()
'''FLIP ALONG X-AXIS'''

In [13]:
taus_boundary = [tau_inner[int(boundary[it,1])] for it in range(ntau)]

Solve $\chi_{II} = 0$ analytically

In [14]:
import scipy.linalg as linalg
## find the tau_zero for response_inh = 0 
def find_cross_paradoxical(x,tau_outer,N,NE,NI,J,hat_sigmae,hat_sigmai,ALPHAE,ALPHAI):
    Z2E = N*J**2*hat_sigmae**2*tau_outer*ALPHAE-N*g*J**2*hat_sigmae*hat_sigmai*x*ALPHAI
    Z2I = -N*g*J**2*hat_sigmae*hat_sigmai*x*ALPHAE+N*g**2*J**2*hat_sigmai**2*tau_outer*ALPHAI
    z2 = np.zeros((N,N))
    z2[:,:NE], z2[:,NE:] = Z2E, Z2I
    J_EFF = Jbar.copy()+z2.copy()
    # print(J_EFF[0,0],J_EFF[NE,NE],J_EFF[0,NE],J_EFF[NE,0])
    
    response_mtx = linalg.inv(np.eye(N)-J_EFF)
    # print('response:',np.sum(response_mtx[NE,NE:]),np.sum(response_mtx[NE+10,NE:]))
    return np.sum(response_mtx[NE,NE:])

tau_x = np.zeros(ntau)
tau_ss = np.linspace(-0.15,0.15,100)[::-1]
for it, tau_outer in enumerate(tau_series):
    prev_x = 100
    for iti, tau_inner in enumerate(tau_ss):
        x = find_cross_paradoxical(tau_inner,tau_outer,N,NE,NI,J,hat_sigmae,hat_sigmai,ALPHAE,ALPHAI)
        if prev_x>0 and x<0:
            tau_x[it] = tau_inner
            print('tau_outer:',tau_outer,'tau_inner:',tau_inner)
            break
        prev_x = x
        

tau_outer: 0.0 tau_inner: -0.06212121212121212
tau_outer: 0.016666666666666666 tau_inner: -0.04999999999999999
tau_outer: 0.03333333333333333 tau_inner: -0.04090909090909091
tau_outer: 0.05 tau_inner: -0.02878787878787878
tau_outer: 0.06666666666666667 tau_inner: -0.016666666666666663
tau_outer: 0.08333333333333333 tau_inner: -0.007575757575757569
tau_outer: 0.1 tau_inner: 0.004545454545454547
tau_outer: 0.11666666666666667 tau_inner: 0.016666666666666663
tau_outer: 0.13333333333333333 tau_inner: 0.025757575757575757
tau_outer: 0.15 tau_inner: 0.03787878787878787


tau_2d[14],tau_2d[13],tau_2d[13],tau_2d[12],tau_2d[11],tau_2d[11],tau_2d[9],tau_2d[8],tau_2d[8],tau_2d[7]
(-0.06666666666666667,
 -0.05,
 -0.05,
 -0.03333333333333333,
 -0.016666666666666666,
 -0.016666666666666666,
 0.0,
 0.016666666666666666,
 0.016666666666666666,
 0.03333333333333333)

 [[ 0. 14.]
 [ 0. 13.]
 [ 0. 13.]
 [ 0. 12.]
 [ 0. 11.]
 [ 0. 11.]
 [ 0.  9.]
 [ 0.  8.]
 [ 0.  8.]
 [ 0.  7.]]

In [15]:
tau_2d = np.zeros(ntau*2-1)
tau_2d[:ntau] = [-tau for tau in tau_series[::-1]]
tau_2d[ntau:] = tau_series[1:]
X,Y = np.meshgrid(tau_2d,tau_2d)

In [16]:
import matplotlib
import matplotlib.cm as cm
fig,ax = plt.subplots(figsize=(7,6))
cmap = cm.get_cmap('coolwarm')
norm = matplotlib.colors.Normalize(vmin=-1.0, vmax=1.0)
ax.pcolormesh(tau_series,tau_series,np.squeeze(response_func_eff[:,:ntau,1]).T,cmap=cmap,norm=norm)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_37916\2383221811.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')


Eq(58) $\lambda_{EE}^{eff}=1$

In [17]:
## using Esubnet instability 
tauc_cross = ((1-NE*c*J)/ALPHAE**2/N**2/J**2/c/(1-c) - tau_series)/gamma/g
print(tauc_cross)

[ 0.06026737  0.04915626  0.03804515  0.02693403  0.01582292  0.00471181
 -0.0063993  -0.01751041 -0.02862152 -0.03973263]


[ 0.06026737  0.04915626  0.03804515  0.02693403  0.01582292  0.00471181
 -0.0063993  -0.01751041 -0.02862152 -0.03973263]

In [18]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(-tau_x,tau_series,'gray',label='Response function')
ax.plot(tauc_cross, tau_series,'k',label='E-stability')
ax.scatter(tauc_cross[5]*0.5+tauc_cross[6]*0.5,tau_series[5]*0.5+tau_series[6]*0.5,c='k',label='E-stability',marker = '^',s=100)
ax.set_xticks(tau_2d)
ax.set_yticks(tau_series)
ax.set_xlim([-0.15,0.15])
ax.set_ylim([0,0.15])
ax.set_aspect('equal')
plt.show()